In [ ]:
import matplotlib.pyplot as plt
import polars as pl

import stock

In [ ]:
# william oneilの手法の再現性を検証する

In [ ]:
code = "6254"
daily_csv_path = stock.PROJECT_ROOT / "data" / "daily" / f"{code}.csv"
weekly_csv_path = stock.PROJECT_ROOT / "data" / "financial" / f"{code}.csv"

ddf = stock.kabutan.read_data_csv(daily_csv_path)
fdf = stock.kabutan.read_financial_csv(weekly_csv_path)

In [ ]:
qfdf = fdf.filter(pl.col("duration") == 3).sort(pl.col("annoounce_date")).with_columns(
    pl.col("eps").rolling_sum(window_size=4).alias("eps12")
)
ddf = ddf.filter(pl.col("date") >= qfdf["annoounce_date"][0])

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(qfdf["annoounce_date"], qfdf["eps12"])
ax1.grid(which="major")
ax1.grid(which="minor", alpha=0.5)

import matplotlib.ticker as ticker
ax1.grid(which='major', lw=0.7) # 主目盛の描画(標準)

# X,Y軸に対して、(補助目盛)×5 = (主目盛)
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(5))
ax1.grid(which='minor', lw=0.4) # 補助目盛の描画
ax1.tick_params(axis="x", labelrotation=90)

ax2 = ax1.twinx()
ax2.plot(ddf["date"], ddf["close"], "red")
ax2.set_yscale("log")

# ax3 = ax1.twinx()
# ax3.plot(ddf["date"], ddf["rs_nikkei"], "green")

In [ ]:
pl.Config.set_tbl_rows(-1)
print(qfdf)
pl.Config.set_tbl_rows(10)